In [2]:
import os, time, re, random, glob, json, jieba
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

In [3]:
device="cuda:0"
# device="cpu"

In [4]:
CLEAN_TEXT_PATTERN = re.compile(r"[\r\n]")

def clean_text(text):
    return CLEAN_TEXT_PATTERN.sub("", text)

# SFT

In [3]:
import sentencepiece
model_file = "/Users/zeyesun/Documents/Data/models/pangu-350M/vocab.model"
sp = sentencepiece.SentencePieceProcessor()
sp.Load(model_file=model_file)

True

In [4]:
for i in range(10):
    print(sp.id_to_piece(i))

<unk>
<s>
</s>
▃
▂
<sep>
<pad>
<mask>
<eod>
<eot>


In [ ]:
model_name_or_path = "D:\\Data\\models\\pangu_2_6B"
# model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, use_cache=False)
model.to(device)
# print(model.device)

In [ ]:
# st = torch.load(os.path.join(model_name_or_path, "pytorch_model.bin"), map_location="cpu")
# print(len(st))
# # st.keys()

In [ ]:
text_generator = TextGenerationPipeline(model, tokenizer, device=device)
# greedy search
print(text_generator("中国和美国和日本和法国和加拿大和澳大利亚的首都分别是哪里？", max_length=50))

### Data Processing

In [6]:
# f = "/Users/zeyesun/Documents/Data/raw/baike_qa2019/baike_qa_train.json"
f = "D:\\Data\\raw\\baike_qa_train.json"
items = []
lens_prompt = []
lens_label = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        items.append(item)
        lens_prompt.append(len(prompt))
        lens_label.append(len(label))
print(len(items))
print(np.percentile(lens_prompt, np.arange(90, 101)))
print(np.percentile(lens_label, np.arange(90, 101)))

1425170


In [43]:
i = 789
items[i]

{'qid': 'qid_7770627748113178417',
 'category': '生活-保健养生',
 'title': '减肥健身我要下载一套很好的有氧健身操，请问哪里有下载地址，谢谢！ ',
 'desc': '我要下载一套很好的有氧操，请问哪里有下载地址，谢谢！',
 'answer': '我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'}

In [44]:
items[i]['answer']

'我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'

# Reward Model

In [9]:
model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)
tokenizer.add_special_tokens({'unk_token': "<unk>",
                                  'bos_token': "<s>",
                                  'eos_token': "<eot>",
                                  'pad_token': "<pad>",
                                  "sep_token": "<sep>"})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


0

In [10]:
max_length = 1024
text = "你好，你是谁"
# text = "<|startoftext|>" + text + "<|endoftext|>"
res = tokenizer(text, max_length=max_length, truncation="longest_first", 
          return_tensors="pt", add_special_tokens=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SUNZEY~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.501 seconds.
Prefix dict has been built successfully.


In [19]:
res.keys()
# torch.cat((res['input_ids'], res['input_ids']), axis=1)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])